In [4]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [54]:
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(25,25,25),)
print(len(satoms["positions"]))
sys = pc.System()
sys.box = box
sys.atoms = satoms


31250


In [55]:
%%timeit
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6, cells=False)

IOStream.flush timed out
IOStream.flush timed out


19 s ± 32.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


IOStream.flush timed out


In [56]:
%%timeit
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6, cells=True)

378 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
